In [1]:
import requests

In [8]:
BASE = "https://api.odcloud.kr/api/gov24/v1/serviceList"
params = {
    "page": 1,
    "perPage": 10,            # 최소 10 권장
    "returnType": "JSON", 
    "serviceKey": "/P5AhAmvjuBQBRALCoeppdhwG2TdfyZiiQNcc+Zkkbdec3MKMsgRs3GZUa+PuCCtdEsTd7DtbbgmLjCTitOhSA=="
}
DEC_KEY = "	%2FP5AhAmvjuBQBRALCoeppdhwG2TdfyZiiQNcc%2BZkkbdec3MKMsgRs3GZUa%2BPuCCtdEsTd7DtbbgmLjCTitOhSA%3D%3D"
try:
    res = requests.get(BASE, params=params, headers={"Accept": "application/json"}, timeout=20)
    print("Request URL:", res.url)               # 실제 전송된 쿼리 확인
    res.raise_for_status()
except requests.HTTPError:
    print("Status:", res.status_code)
    print("Body :", res.text[:800])              # 서버가 준 에러 메시지 확인
    raise ""

data = res.json()
rows = data.get("data", data) if isinstance(data, dict) else data
print("총 건수 추정:", data.get("totalCount", "알수없음") if isinstance(data, dict) else "알수없음")
print("샘플 1건 키:", list(rows[0].keys()) if rows else [])
print("샘플 1건:", rows[0] if rows else {})

Request URL: https://api.odcloud.kr/api/gov24/v1/serviceList?page=1&perPage=10&returnType=JSON&serviceKey=%2FP5AhAmvjuBQBRALCoeppdhwG2TdfyZiiQNcc%2BZkkbdec3MKMsgRs3GZUa%2BPuCCtdEsTd7DtbbgmLjCTitOhSA%3D%3D
Status: 400
Body : {"code":-3,"msg":"등록되지 않은 서비스 입니다."}



TypeError: exceptions must derive from BaseException

In [5]:
ENC_KEY = "%2FP5AhAmvjuBQBRALCoeppdhwG2TdfyZiiQNcc%2BZkkbdec3MKMsgRs3GZUa%2BPuCCtdEsTd7DtbbgmLjCTitOhSA%3D%3D"
url = (
    "https://api.odcloud.kr/api/gov24/v1/serviceList"
    f"?page=1&perPage=10&returnType=JSON&serviceKey={ENC_KEY}"
)

res = requests.get(url, timeout=20, headers={"Accept": "application/json"})
print("URL:", res.url)
print("Status:", res.status_code)
print("Body:", res.text[:500])
res.raise_for_status()

URL: https://api.odcloud.kr/api/gov24/v1/serviceList?page=1&perPage=10&returnType=JSON&serviceKey=%2FP5AhAmvjuBQBRALCoeppdhwG2TdfyZiiQNcc%2BZkkbdec3MKMsgRs3GZUa%2BPuCCtdEsTd7DtbbgmLjCTitOhSA%3D%3D
Status: 400
Body: {"code":-3,"msg":"등록되지 않은 서비스 입니다."}



HTTPError: 400 Client Error: Bad Request for url: https://api.odcloud.kr/api/gov24/v1/serviceList?page=1&perPage=10&returnType=JSON&serviceKey=%2FP5AhAmvjuBQBRALCoeppdhwG2TdfyZiiQNcc%2BZkkbdec3MKMsgRs3GZUa%2BPuCCtdEsTd7DtbbgmLjCTitOhSA%3D%3D

In [6]:
url = f"https://api.odcloud.kr/api/gov24/v1/serviceList?page=1&perPage=10&returnType=JSON&serviceKey={ENC_KEY}"
r = requests.get(url, headers={"Accept":"application/json"}, timeout=15)
print(r.status_code, r.text[:500])

400 {"code":-3,"msg":"등록되지 않은 서비스 입니다."}



In [9]:

def service_list(page=1, per_page=10, extra=None):
    url = f"{BASE}/v3/serviceList"
    params = {"page": page, "perPage": per_page, "serviceKey": DEC_KEY, "returnType": "JSON"}
    if extra: params.update(extra)          # 예: {"cond[svcNm::LIKE]": "청년"}
    r = requests.get(url, params=params, timeout=20, headers={"Accept": "application/json"})
    r.raise_for_status()
    j = r.json()
    rows = j.get("data", j)
    return rows, j.get("totalCount")

def service_detail(svc_id):
    url = f"{BASE}/v3/serviceDetail"        # v3 우선
    r = requests.get(url, params={"svcId": svc_id, "serviceKey": DEC_KEY, "returnType": "JSON"}, timeout=20)
    r.raise_for_status()
    return r.json()

def support_conditions(svc_id):
    # 보조 API도 v3 기준. (혹시 404 등 뜨면 v1로 한 번 더 시도)
    url = f"{BASE}/v3/supportConditions"
    r = requests.get(url, params={"svcId": svc_id, "serviceKey": DEC_KEY, "returnType": "JSON"}, timeout=20)
    if r.status_code != 200:
        url = f"{BASE}/v1/supportConditions"
        r = requests.get(url, params={"svcId": svc_id, "serviceKey": DEC_KEY, "returnType": "JSON"}, timeout=20)
    r.raise_for_status()
    return r.json()

# 1) 목록 10건만 확인
rows, total = service_list(page=1, per_page=10)
print("total:", total, "rows:", len(rows))
print("첫 건 키:", list(rows[0].keys()))
print("샘플 svcId / svcNm:", rows[0]["svcId"], rows[0]["svcNm"])

# 2) 상세/지원조건 확인
svc_id = rows[0]["svcId"]
detail = service_detail(svc_id)
conds  = support_conditions(svc_id)
print("detail keys:", list(detail.keys()))
print("support keys:", list(conds.keys()))

# 3) 여러 페이지 덤프 → CSV
def dump_csv(max_pages=3, per_page=100, path="gov24_serviceList.csv"):
    bucket = []
    for p in range(1, max_pages+1):
        page_rows, _ = service_list(page=p, per_page=per_page)
        if not page_rows: break
        bucket.extend(page_rows)
    pd.DataFrame(bucket).to_csv(path, index=False, encoding="utf-8-sig")
    return path

HTTPError: 404 Client Error: Not Found for url: https://api.odcloud.kr/api/gov24/v1/serviceList/v3/serviceList?page=1&perPage=10&serviceKey=%09%252FP5AhAmvjuBQBRALCoeppdhwG2TdfyZiiQNcc%252BZkkbdec3MKMsgRs3GZUa%252BPuCCtdEsTd7DtbbgmLjCTitOhSA%253D%253D&returnType=JSON